In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
next_layer = "silver"

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")


In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
# Dynamic select query to read the data for specific proc_date
dynamic_query = f""" CREATE OR REPLACE TEMP VIEW temp_pos_wcm_saleout AS SELECT *
FROM udp_wcm_pro.udp_wcm_bronze_pro.pos_wcm_saleout
WHERE ProcDate like '{str(proc_date).replace('-', '')}%'
"""

# Execute the query
spark.sql(dynamic_query)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_realtime.pos_wcm_saleout_bill_props
(
hash_id BIGINT,
receipt_no STRING,
calday DATE,
prop_name STRING,
prop_value STRING,
proc_date TIMESTAMP
) 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'true',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'true'
)
""")

In [0]:
spark.sql(f"""
CREATE OR REPLACE TEMP VIEW view_pos_wcm_saleout_bill_props as
WITH dat AS (
    SELECT
      LTRIM('0', main_tbl.ReceiptNo) AS receipt_no,
      DATE(TO_TIMESTAMP(main_tbl.calendarday, 'MM/dd/yyyy HH:mm:ss')) AS calday,
      orderinfo_tbl.key AS prop_name,  
      orderinfo_tbl.value AS prop_value, 
      TO_TIMESTAMP(main_tbl.ProcDate, 'yyyyMMddHHmmss') AS proc_date
    FROM temp_pos_wcm_saleout main_tbl
    LATERAL VIEW EXPLODE(main_tbl.orderInfo) AS orderinfo_tbl 
  )
  SELECT
    farm_fingerprint(CONCAT(
      IFNULL(CAST(receipt_no AS STRING), "")
    )) AS hash_id,
    *
  FROM dat
  QUALIFY ROW_NUMBER() OVER (PARTITION BY receipt_no ORDER BY proc_date DESC) = 1;    
""")

In [0]:
spark.sql(f"DELETE FROM {catalog_name}.udp_wcm_silver_realtime.pos_wcm_saleout_bill_props WHERE RECEIPT_NO IN (SELECT RECEIPT_NO FROM view_pos_wcm_saleout_bill_props)")

In [0]:
spark.sql(f"INSERT INTO {catalog_name}.udp_wcm_silver_realtime.pos_wcm_saleout_bill_props SELECT * FROM view_pos_wcm_saleout_bill_props")